<a href="https://colab.research.google.com/github/BRIJNANDA1979/BIGEARTHNET/blob/main/BigEarthNet_CNN_for_uniquely_labelled_46_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BigEarthNet unique labelled data with 46 classes from data uploaded on google drive.

Step  1: to create geotiff image using 12 bands of Sentinel-2 imagery 
Step 2 :  to create batch of data with labels
Step 3: Normalize data
Step 4: Transform data in proper shape
Step 5: Data input to CNN
Step 6: Reslut and prediction of data


In [47]:
import os
import pandas as pd

In [48]:
src_path = '/content/drive/MyDrive/BigEarthNet unique labelled'

In [49]:
!pip install rasterio

In [50]:
!pip install glob2

In [51]:
import rasterio
from rasterio import merge
import glob2
def merge_bands(path,sub):
    search = '*_B*.tif'   # 12 Band files end with B01,B02....
    files_batch = []
    files_batch = glob2.glob(os.path.join(path,search)) #search all tif band files 
    files_batch =sorted(files_batch)
    print(files_batch) 
    print(len(files_batch))
    if len(files_batch) == 0 or len(files_batch) > 12 :
      print("Some folders are empty or already contain merged geotiff file and dont have 12 band files")
      return
    # Read metadata of first file
    with rasterio.open(files_batch[0]) as src0: #open first files out of batch of 12 files
        meta = src0.meta

    # Update meta to reflect the number of layers in merged output geotiff file
    meta.update(count = len(files_batch)) #change output meta files as it will have 12 bands rather than 1 as in source files

    # Read each layer and write it to stack
    fnames = path +'/' + sub + '.tif'
    with rasterio.open(fnames, 'w', **meta) as dst: #output/destination(dst) file opened in write mode with an extra "meta" arbitrary keyword argument
        print(dst)
        for id, layer in enumerate(files_batch, start=1): # create list of 12 tuples 
            print(id,layer)
            with rasterio.open(layer) as src1:
                dst.write_band(id, src1.read(1)) #read 1 band at a time and write it to stacled tif

In [52]:
dirs = os.listdir(src_path)  #list of all classes
print(dirs)
print(len(dirs))

['Agro-forestry areas', 'Airports', 'Annual crops associated with permanent crops', 'Coastal lagoons', 'Complex cultivation patterns', 'Discontinuous urban fabric', 'Dump sites', 'Estuaries', 'Fruit trees and berry plantations', 'Green urban areas', 'Industrial or commercial units', 'Inland marshes', 'Intertidal flats', 'Mineral extraction sites', 'Mixed forest', 'Moors and heathland', 'Natural grassland', 'Permanently irrigated land', 'Rice fields', 'Salt marshes', 'Sclerophyllous vegetation', 'Sparsely vegetated areas', 'Sport and leisure facilities', 'Transitional woodlandshrub', 'Vineyards']
25


In [53]:
# TESTING the input data..before merging 12 bands to geotiff
%%timeit
import glob2 # glob2 is faster than glob
for dir in dirs:
  sub_dirs = os.listdir(os.path.join(src_path,dir))
  src_folders_path = os.path.join(src_path,dir)
  for sub in sub_dirs:
    path=os.path.join(src_folders_path,sub)
    #print(len(sub_dirs))
    search = '*_B*.tif'   # 12 Band files end with B01,B02....
    files_batch = []
    files_batch = glob2.glob(os.path.join(path,search)) #search all tif band files 
    files_batch =sorted(files_batch)
    #print(files_batch) 
    if len(files_batch) < 12:
      print(len(files_batch))
      print(dir)
      exit_parameter  = True
      break
    else:
      continue
    if exit_parameter == True:
      break

1 loop, best of 5: 1.79 s per loop


In [54]:
%%timeit
import glob2 # glob2 is faster than glob

for dir in dirs:
  sub_dirs = os.listdir(os.path.join(src_path,dir))
  path=os.path.join(src_path,dir)
  print(dir,len(sub_dirs))
  for data_folder in sub_dirs:
    path_tif_files=os.path.join(path,data_folder)
    merge_bands(path_tif_files,data_folder)

Streaming output truncated to the last 5000 lines.
10 /content/drive/MyDrive/BigEarthNet unique labelled/Transitional woodlandshrub/S2B_MSIL2A_20180326T112109_80_66/S2B_MSIL2A_20180326T112109_80_66_B11.tif
11 /content/drive/MyDrive/BigEarthNet unique labelled/Transitional woodlandshrub/S2B_MSIL2A_20180326T112109_80_66/S2B_MSIL2A_20180326T112109_80_66_B12.tif
12 /content/drive/MyDrive/BigEarthNet unique labelled/Transitional woodlandshrub/S2B_MSIL2A_20180326T112109_80_66/S2B_MSIL2A_20180326T112109_80_66_B8A.tif
['/content/drive/MyDrive/BigEarthNet unique labelled/Transitional woodlandshrub/S2B_MSIL2A_20180326T112109_28_62/S2B_MSIL2A_20180326T112109_28_62_B01.tif', '/content/drive/MyDrive/BigEarthNet unique labelled/Transitional woodlandshrub/S2B_MSIL2A_20180326T112109_28_62/S2B_MSIL2A_20180326T112109_28_62_B02.tif', '/content/drive/MyDrive/BigEarthNet unique labelled/Transitional woodlandshrub/S2B_MSIL2A_20180326T112109_28_62/S2B_MSIL2A_20180326T112109_28_62_B03.tif', '/content/drive/My

In [55]:
import pandas as pd
src_path='/content/drive/MyDrive/BigEarthNet unique labelled'
classes = os.listdir(src_path)
classes

['Agro-forestry areas',
 'Airports',
 'Annual crops associated with permanent crops',
 'Coastal lagoons',
 'Complex cultivation patterns',
 'Discontinuous urban fabric',
 'Dump sites',
 'Estuaries',
 'Fruit trees and berry plantations',
 'Green urban areas',
 'Industrial or commercial units',
 'Inland marshes',
 'Intertidal flats',
 'Mineral extraction sites',
 'Mixed forest',
 'Moors and heathland',
 'Natural grassland',
 'Permanently irrigated land',
 'Rice fields',
 'Salt marshes',
 'Sclerophyllous vegetation',
 'Sparsely vegetated areas',
 'Sport and leisure facilities',
 'Transitional woodlandshrub',
 'Vineyards']

In [68]:
data=[]
labels=[]
code=[]
code_var=0
for files in classes:
  geotiff_files = os.listdir(os.path.join(src_path,files))
  #print(geotiff_files)
  print(len(geotiff_files))
  for i in geotiff_files:
    data.append(i)
    labels.append(files)
    code.append(code_var)
  code_var = code_var+1
  
data
labels
code

666
3
19
268
81
87
3
105
60
2
4
30
24
6
449
20
6
156
12
8
80
6
4
389
26


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [69]:
df=pd.DataFrame()
df['data'] = data
df['labels'] = labels
df['code'] = code
df.head()
df.describe()

,code
count,2514.000000
mean,9.816229
std,8.573344
min,0.000000
25%,0.000000
50%,8.000000
75%,17.000000
max,24.000000


In [70]:
df[0:670]
df.shape
#df.size

(2514, 3)

In [82]:
from rasterio.plot import reshape_as_raster, reshape_as_image
from rasterio.plot import adjust_band
#print(files) #geotiff files

fpath = '/content/drive/MyDrive/BigEarthNet unique labelled'

batch=[]
for row in df.itertuples():
  f = fpath + '/' + row[2] + '/' + row[1] + '/' + row[1] + '.tif'
  img_data = rasterio.open(f)
  image = img_data.read()
  image_norm = adjust_band(image) # normalize bands to range between 1.0 to 0.0
  image_reshaped = reshape_as_image(image_norm) # reshape to [rows, cols, bands]
  print(image_reshaped.shape)
  print(img_data.shape)
  batch.append(image_reshaped)
print(len(batch))
batch=np.asarray(batch)
print(batch.shape)
X=batch    # Input for CNN

Streaming output truncated to the last 5000 lines.
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20, 20, 12)
(20, 20)
(20

In [72]:
print(len(classes))
l=len(classes)
code_classes = list(enumerate(classes,0))   # code the classes
code_classes

25


[(0, 'Agro-forestry areas'),
 (1, 'Airports'),
 (2, 'Annual crops associated with permanent crops'),
 (3, 'Coastal lagoons'),
 (4, 'Complex cultivation patterns'),
 (5, 'Discontinuous urban fabric'),
 (6, 'Dump sites'),
 (7, 'Estuaries'),
 (8, 'Fruit trees and berry plantations'),
 (9, 'Green urban areas'),
 (10, 'Industrial or commercial units'),
 (11, 'Inland marshes'),
 (12, 'Intertidal flats'),
 (13, 'Mineral extraction sites'),
 (14, 'Mixed forest'),
 (15, 'Moors and heathland'),
 (16, 'Natural grassland'),
 (17, 'Permanently irrigated land'),
 (18, 'Rice fields'),
 (19, 'Salt marshes'),
 (20, 'Sclerophyllous vegetation'),
 (21, 'Sparsely vegetated areas'),
 (22, 'Sport and leisure facilities'),
 (23, 'Transitional woodlandshrub'),
 (24, 'Vineyards')]

In [75]:
#convert list to array
import numpy as np
batch=np.asarray(batch)
print(batch.shape)
y=df['code']
print(y)
y=np.array(y)
print(y)

(2514, 20, 20, 12)
0        0
1        0
2        0
3        0
4        0
        ..
2509    24
2510    24
2511    24
2512    24
2513    24
Name: code, Length: 2514, dtype: int64
[ 0  0  0 ... 24 24 24]


In [84]:
print(X.shape)    # data is ready for CNN
print(y.shape)


(2514, 20, 20, 12)
(2514,)


In [85]:
print(batch[0])  #first tiff image normalized

[[[0.00860547 0.04721977 0.10105914 ... 0.41240071 0.21690203 0.52206531]
  [0.0178729  0.02669903 0.06443071 ... 0.40269197 0.22043248 0.47352162]
  [0.03353928 0.04589585 0.08451015 ... 0.50242718 0.33671668 0.44417476]
  ...
  [0.03508385 0.06840247 0.11915269 ... 0.63195057 0.42012357 0.60834069]
  [0.02714034 0.070609   0.1299647  ... 0.69086496 0.45609003 0.64298323]
  [0.00838482 0.03971756 0.11142983 ... 0.44505737 0.21293027 1.        ]]

 [[0.00948808 0.03993822 0.08892321 ... 0.4190203  0.20631068 0.54104148]
  [0.01125331 0.02802295 0.07105031 ... 0.38106796 0.20476611 0.47992056]
  [0.02934687 0.06421006 0.11032657 ... 0.5348632  0.34488085 0.51897617]
  ...
  [0.03243601 0.07281553 0.12775816 ... 0.61142983 0.43998235 0.52360989]
  [0.02427184 0.05538394 0.0988526  ... 0.58031774 0.38526037 0.47992056]
  [0.00397176 0.02736099 0.07480141 ... 0.41085613 0.21646072 0.63570168]]

 [[0.00794351 0.03817299 0.08958517 ... 0.41857899 0.20895852 0.60635481]
  [0.00617829 0.025154

In [86]:
!pip install keras.utils


Processing /root/.cache/pip/wheels/46/25/27/7707005c1cb27e1ffc8277b004ac295e34767b02b44d73d6be/keras_utils-1.0.13-cp37-none-any.whl


In [87]:
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)
y_train = np_utils.to_categorical(y_train, len(classes))
y_test = np_utils.to_categorical(y_test, len(classes))

(1684, 20, 20, 12)
(830, 20, 20, 12)
(830,)
(1684,)


In [91]:
print(y_train[1681:1683])  # visualize y_train data

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]]


In [92]:
input_shape = (20,20,12)
num_classes = len(classes)

In [93]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint

In [94]:
model = keras.models.Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 20, 32)        3488      
_________________________________________________________________
batch_normalization (BatchNo (None, 20, 20, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 20, 20, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 10, 10, 64)        0

In [95]:
model.compile(
   loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']
)

In [96]:
from keras.callbacks import EarlyStopping 

In [97]:
history = model.fit(X_train, y_train, batch_size = 30, epochs = 150, validation_data = (X_test, y_test),
                    callbacks = [EarlyStopping(monitor = 'val_loss', patience = 20)])

Epoch 1/150
57/57 [==============================] - 26s 109ms/step - loss: 2.4604 - accuracy: 0.3618 - val_loss: 2.5561 - val_accuracy: 0.2747
Epoch 2/150
57/57 [==============================] - 5s 90ms/step - loss: 1.6029 - accuracy: 0.5277 - val_loss: 2.6010 - val_accuracy: 0.2855
Epoch 3/150
57/57 [==============================] - 5s 89ms/step - loss: 1.3851 - accuracy: 0.5835 - val_loss: 2.1008 - val_accuracy: 0.4060
Epoch 4/150
57/57 [==============================] - 5s 90ms/step - loss: 1.1796 - accuracy: 0.6296 - val_loss: 2.7372 - val_accuracy: 0.3831
Epoch 5/150
57/57 [==============================] - 5s 89ms/step - loss: 1.3099 - accuracy: 0.6113 - val_loss: 1.5034 - val_accuracy: 0.5687
Epoch 6/150
57/57 [==============================] - 5s 88ms/step - loss: 1.1102 - accuracy: 0.6684 - val_loss: 1.3077 - val_accuracy: 0.5771
Epoch 7/150
57/57 [==============================] - 5s 89ms/step - loss: 1.1092 - accuracy: 0.6449 - val_loss: 1.2209 - val_accuracy: 0.6096
Epoc

In [99]:
score = model.evaluate(X_test, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 1.754902720451355
Test accuracy: 0.6361445784568787


In [102]:
pred = model.predict(X_test) 
pred = np.argmax(pred, axis = 1)[:100] 
label = np.argmax(y_test,axis = 1)[:100] 

print(pred) 
print(label)

[ 0  3  3  0 17 24 14  0  0  0  0 23  0  0 11  7  0 17 14 23 14  3  0 20
  8 15 14  0  3  0  7 11 11 15  0  7 14  0  5 15 11 23 14  0 23  0 11  0
 24  3  3 23 19  0  3  0  0 14  0 23 15  0 14  0 23 14  0  0  7 15  0  0
 11  0 23  0  7 14  0  5  0  0 23 14  0  0  0  0  0 15 14  0  0 23  4 15
 11  0  3  0]
[ 0  3  3  4 17  5 14  0 20  0  4 23  0  0 14  7  0 17 14  5 14  3  0 20
  4 20 14 23  3  0  7  0 14 14  0  7 14  0  5 14 14 23 14  0  4 23 14  0
  5  3  3 23 19  4  3  0  0 14 23 18 23  4 14 23 23 14 20  8  7 14  0  0
  0  0  4  0  7 14  8  5  0 23 23 14  0  0  0  0 23 20 14  0  0 23 17  4
 11  0  3  0]


In [101]:
prediction = model.predict(X_test)
print(prediction.flatten()) 

[9.9945563e-01 8.7520220e-06 1.9651583e-05 ... 1.6991672e-07 2.0630662e-04
 1.6895125e-07]
